In [17]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from aim import Run

In [20]:
from surprise import BaselineOnly, Dataset, Reader
from surprise.model_selection import cross_validate
from surprise import accuracy, Dataset
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVD, SVDpp, NMF
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline

In [66]:
reader = Reader(line_format="user item rating timestamp", sep="\t")
data = Dataset.load_from_file("data/u.data", reader=reader)
trainset, testset = train_test_split(data, test_size=0.2)

## NMF

In [55]:
aim_run = Run(
    experiment='movie_recs_surprise'
)

n_factors=25
n_epochs=50
biased=False
reg_pu=0.06
reg_qi=0.06
reg_bu=0.02
reg_bi=0.02
lr_bu=0.005
lr_bi=0.005

# Log run parameters
aim_run["hparams"] = {
    'algo': 'NMF',
    'n_factors': n_factors,
    'biased': biased,
    'reg_pu': reg_pu,
    'reg_qi':reg_qi,
    'reg_bu': reg_bu,
    'reg_bi': reg_bi,
    'lr_bu': lr_bu,
    'lr_bi': lr_bi
}

In [56]:
for epoch in tqdm(range(n_epochs)):
    algo = NMF(n_factors=n_factors, 
               n_epochs=epoch, 
               biased=biased, 
               reg_pu=reg_pu, 
               reg_qi=reg_qi, 
               reg_bu=reg_bu, 
               reg_bi=reg_bi, 
               lr_bu=lr_bu, 
               lr_bi=lr_bi, 
               random_state=None, 
               verbose=False)
    algo.fit(trainset)
    predictions = algo.test(testset)

    rmse = accuracy.rmse(predictions, verbose=False)
    fcp = accuracy.fcp(predictions, verbose=False)

    aim_run.track({'rmse': rmse, 'fcp': fcp}, context={'subset': 'test'}, epoch=epoch)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:54<00:00,  2.28s/it]


## SVD

In [63]:
n_factors=150
n_epochs=50 
biased=True
init_mean=0
init_std_dev=0.1
lr_all=0.005
reg_all=0.02
lr_bu=None
lr_bi=None
lr_pu=None
lr_qi=None
reg_bu=None
reg_bi=None
reg_pu=None
reg_qi=None

aim_run = Run(
    experiment='movie_recs_surprise'
)

aim_run["hparams"] = {
    'algo': 'SVD',
    'n_factors': n_factors,
    'biased': biased,
    'init_mean': init_mean,
    'init_std_dev': init_std_dev,
    'lr_all': lr_all,
    'reg_all': reg_all,
    'lr_bu': lr_bu,
    'lr_bi': lr_bi,
    'lr_pu': lr_pu,
    'lr_qi': lr_qi,
    'reg_bu': reg_bu,
    'reg_bi': reg_bi,
    'reg_pu': reg_pu,
    'reg_qi': reg_qi
}

In [64]:
for epoch in tqdm(range(n_epochs)):
    algo = SVD(n_factors=n_factors, 
               n_epochs=epoch, 
               biased=biased, 
               init_mean=init_mean,
               init_std_dev=init_std_dev,
               lr_all=lr_all,
               reg_all=reg_all,
               lr_bu=lr_bu,
               lr_bi=lr_bi,
               lr_pu=lr_pu,
               lr_qi=lr_qi,
               reg_bu=reg_bu,
               reg_bi=reg_bi,
               reg_pu=reg_pu,
               reg_qi=reg_qi,
               random_state=None, 
               verbose=False)
    algo.fit(trainset)
    predictions = algo.test(testset)

    rmse = accuracy.rmse(predictions, verbose=False)
    fcp = accuracy.fcp(predictions, verbose=False)

    aim_run.track({'rmse': rmse, 'fcp': fcp}, context={'subset': 'test'}, epoch=epoch)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [06:00<00:00,  7.21s/it]
